# Table of contents

* [Setup](#Setup)
  * [Module imports](#Module-imports)
  * [Load spark session and data](#Load-spark-session-and-data)
  * [Declare globals](#Declare-globals)
* [Distribution of values](#Distribution-of-values)

# Setup

## Module imports

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, median, mode, skewness, split, when
import importlib
import numpy as np

In [2]:
from modules.load import read_file
from modules.plots import catplot, countplot, display_images, distribution

## Load spark session and data

In [3]:
spark = SparkSession.builder.appName("Test").getOrCreate()

In [4]:
df = read_file(spark, "data/transactions.ndjson")

In [5]:
# Clean up the "merchantName" column
df = df.withColumn(
    "merchantName",
    when(col("merchantName").contains(" #"), split(col("merchantName"), " #").getItem(0))
                            .when(col("merchantName").contains("Blue Mountain"), "Blue Mountain")
                            .when(col("merchantName").contains("ethanallen.com"), "Ethan Allen")
                            .when(col("merchantName").contains("pottery-barn.com"), "Pottery Barn")
                            .when(col("merchantName").contains("westelm.com"), "West Elm")
                            .when(col("merchantName").contains("williamssonoma"), "Williams Sonoma")
                            .otherwise(col("merchantName"))
)

## Declare globals

All the column names in the dataset shown for reference

In [6]:
np.array(df.columns).reshape((13,3)).tolist()

[['accountNumber', 'accountOpenDate', 'acqCountry'],
 ['availableMoney', 'cardCVV', 'cardLast4Digits'],
 ['cardPresent', 'creditLimit', 'currentBalance'],
 ['currentExpDate', 'customerId', 'dateOfLastAddressChange'],
 ['enteredCVV', 'expirationDateKeyInMatch', 'isFraud'],
 ['merchantCategoryCode', 'merchantCountryCode', 'merchantName'],
 ['posConditionCode', 'posEntryMode', 'transactionAmount'],
 ['transactionDateTime', 'transactionType', 'creditLimitIndexed'],
 ['merchantNameIndexed', 'acqCountryIndexed', 'merchantCountryCodeIndexed'],
 ['posEntryModeIndexed',
  'posConditionCodeIndexed',
  'merchantCategoryCodeIndexed'],
 ['transactionTypeIndexed', 'creditLimitEncoded', 'merchantNameEncoded'],
 ['acqCountryEncoded', 'merchantCountryCodeEncoded', 'posEntryModeEncoded'],
 ['posConditionCodeEncoded',
  'merchantCategoryCodeEncoded',
  'transactionTypeEncoded']]

### Labels and groups
Assign names to columns grouped by type for labeling purposes

In [7]:
# Define the list of decimal type columns
decimal_cols = [col for col, dtype in df.dtypes if "decimal" in dtype]

# Define the list of numerical type columns
numerical_cols = [col for col, dtype in df.dtypes if ("decimal" in dtype) or ("double" in dtype) or ("int" in dtype)]

# Define the list of categorical columns
categorical_cols = [
    "creditLimit",
    "merchantName",
    "acqCountry",
    "merchantCountryCode",
    "posEntryMode",
    "posConditionCode",
    "merchantCategoryCode",
    "transactionType"
]

catkinds = ["strip", "box", "boxen", "violin"]

# Distribution of values

### Statistical summaries of each variables

For numerical columns, a summary was obtained to get an overview of the statistics. There are some additional information I wanted to see that isn't part of the built in .summary() stats, so I produced them separately as well.

In [21]:
df.select(decimal_cols).summary().show()
for col in decimal_cols:
    df.select(skewness(col), median(col), mode(col)).show()

+-------+-----------------+------------------+-----------------+-----------------+
|summary|   availableMoney|       creditLimit|   currentBalance|transactionAmount|
+-------+-----------------+------------------+-----------------+-----------------+
|  count|           786363|            786363|           786363|           786363|
|   mean|      6250.725369|      10759.464459|      4508.739089|       136.985791|
| stddev|8880.783989147438|11636.174889962065|6457.442068131113| 147.725569019521|
|    min|         -1005.63|            250.00|             0.00|             0.00|
|    25%|          1077.12|            5000.0|           689.72|            33.65|
|    50%|          3184.02|            7500.0|           2451.2|            87.89|
|    75%|           7500.0|           15000.0|          5289.06|           191.44|
|    max|         50000.00|          50000.00|         47498.81|          2011.54|
+-------+-----------------+------------------+-----------------+-----------------+

+--

One thing that stood out to me as being very confusing was that the mode of transactionAmount is 0. I am surprised that transactions of 0 values are even permitted in the first place, let alone for that to be the most common value. I am not sure what to make of this, honestly.

Next, in the case of categorical variables, Things like averages, stddev etc. don't make sense, so I only calculated the modes.

In [23]:
df.select([mode(col) for col in categorical_cols]).show()

+-----------------+------------------+----------------+-------------------------+------------------+----------------------+--------------------------+---------------------+
|mode(creditLimit)|mode(merchantName)|mode(acqCountry)|mode(merchantCountryCode)|mode(posEntryMode)|mode(posConditionCode)|mode(merchantCategoryCode)|mode(transactionType)|
+-----------------+------------------+----------------+-------------------------+------------------+----------------------+--------------------------+---------------------+
|          5000.00|               AMC|              US|                       US|                05|                    01|             online_retail|             PURCHASE|
+-----------------+------------------+----------------+-------------------------+------------------+----------------------+--------------------------+---------------------+



### Single variable visualizations

In [8]:
# Look at the distribution of values of all the numerical columns
for column in decimal_cols:
    continue
    distribution(df.select(column), column, log_scale=True)
    print(column)
for column in categorical_cols:
    continue
    countplot(df.select(column), column)

<div>
<p style="text-align: center;">Plots showing the distributions of the four decimal quantities</p>
<img src="figures/availableMoney_distribution.png" width="250"/>
<img src="figures/creditLimit_distribution.png" width="250"/>
<img src="figures/currentBalance_distribution.png" width="250"/>
<img src="figures/transactionAmount_distribution.png" width="250"/><br>
<p style="text-align: center;">Plots showing the distribution of categorical columns</p>
<img src="figures/creditLimit_barchart.png" width="250"/>
<img src="figures/acqCountry_barchart.png" width="250"/>
<img src="figures/merchantCountryCode_barchart.png" width="250"/>
<img src="figures/posEntryMode_barchart.png" width="250"/><br>
<img src="figures/merchantCategoryCode_barchart.png" width="250"/>
<img src="figures/transactionType_barchart.png" width="250"/>
<img src="figures/merchantName_barchart.png" width="250"/><br>
</div>
Something that is obvious in retrospect that I didn't think about was that credit limit is really a categorical variable even though there is technically nothing preventing a bank assigning any non-negative credit limit to a customer, so I ended up plotting it twice, once treating the values as decimal and again using the values as categorical.

### Single variables, grouped by fraud/not fraud

#### Summary statistic
The same process of generating summary statistics were repeated, but this time grouped by the values of isFraud column.

In [29]:
print("Summary statistic of fraudulent transactions")
df.select(decimal_cols).where(df.isFraud == True).summary().show()
for col in decimal_cols:
    df.where(df.isFraud == True).select(skewness(col), median(col), mode(col)).show()
print("Summary statistic of legitimate transactions")
df.select(decimal_cols).where(df.isFraud == False).summary().show()
for col in decimal_cols:
    df.where(df.isFraud == False).select(skewness(col), median(col), mode(col)).show()

Summary statistic of fraudulent transactions
+-------+-----------------+------------------+-----------------+------------------+
|summary|   availableMoney|       creditLimit|   currentBalance| transactionAmount|
+-------+-----------------+------------------+-----------------+------------------+
|  count|            12417|             12417|            12417|             12417|
|   mean|      6142.894186|      11044.958525|      4902.064338|        225.215905|
| stddev|8703.131117429253|11943.007882894744|7074.701648973073|189.55139322546836|
|    min|          -614.39|            250.00|             0.00|              0.00|
|    25%|          1078.02|            5000.0|           821.75|             85.92|
|    50%|          3120.95|            7500.0|          2747.25|            176.97|
|    75%|           7500.0|           15000.0|          5644.35|            311.21|
|    max|         50000.00|          50000.00|         47473.94|           1608.35|
+-------+-----------------+----

In general, fraudulent transactions have wider spread of values, resulting in higher values of mean, stddev etc., but the two distributions still seem to overlap substantially.

In [9]:
for column in decimal_cols:
    continue
    for kind in catkinds:
        catplot(df.select("isFraud", column), "isFraud", column, kind=kind)

for column in categorical_cols:
    continue
    distribution(df.select("isFraud", col(column).cast("string")), column, hue="isFraud", stat="proportion", common_norm=False)

<div>
<p style="text-align: center;">Box plots of the four decimal columns, grouped by fraud/legit transactions</p>
<img src="figures/availableMoney_boxplot_by_isFraud.png" width="340"/>
<img src="figures/creditLimit_boxplot_by_isFraud.png" width="340"/>
<img src="figures/currentBalance_boxplot_by_isFraud.png" width="340"/>
<img src="figures/transactionAmount_boxplot_by_isFraud.png" width="340"/><br>
<p style="text-align: center;">Boxen plots of the four decimal columns, grouped by fraud/legit transactions</p>
<img src="figures/availableMoney_boxenplot_by_isFraud.png" width="340"/>
<img src="figures/creditLimit_boxenplot_by_isFraud.png" width="340"/>
<img src="figures/currentBalance_boxenplot_by_isFraud.png" width="340"/>
<img src="figures/transactionAmount_boxenplot_by_isFraud.png" width="340"/><br>
<p style="text-align: center;">Strip plots of the four decimal columns, grouped by fraud/legit transactions</p>
<img src="figures/availableMoney_stripplot_by_isFraud.png" width="340"/>
<img src="figures/creditLimit_stripplot_by_isFraud.png" width="340"/>
<img src="figures/currentBalance_stripplot_by_isFraud.png" width="340"/>
<img src="figures/transactionAmount_stripplot_by_isFraud.png" width="340"/><br>
<p style="text-align: center;">Violin plots of the four decimal columns, grouped by fraud/legit transactions</p>
<img src="figures/availableMoney_violinplot_by_isFraud.png" width="340"/>
<img src="figures/creditLimit_violinplot_by_isFraud.png" width="340"/>
<img src="figures/currentBalance_violinplot_by_isFraud.png" width="340"/>
<img src="figures/transactionAmount_violinplot_by_isFraud.png" width="340"/><br>
</div>

1. The values of "availableMoney" variable is very similar between legit/fraud groups.
2. "creditLimit" is a little more widely distributed in fraud group compared to legit group.
3. Greater proportion of legit accounts have "currentBalance" and "transactionAmount" very close to zero compared to fraud group.

In general, the distributions of each quantity are not qualitatively different between fraud and legit transactions, but there are quantitative differences.

## Three-variable analysis
Next I looked at the distribution of numerical variables for the fraud/legit groups, split by categorical variables.

In [10]:
#catplot(df.select("isFraud", "transactionAmount", "transactionType"), "isFraud", "transactionAmount", kind="violin", hue="transactionType")
#Image(filename="figures/transactionAmount_violinplot_by_isFraud_with_transactionType.png")

Above plot shows that non-fraudulent transactions are concentrated very close to zero in all categories, while fraudulent charges are more spread out. This difference is especially notable for unmarked ("empty") transactions. The average size of the fraudulent transactions are consistently higher across all categories as a result.

In [12]:
catplot(df.select("isFraud", "transactionAmount", "merchantCategoryCode"), "isFraud", "transactionAmount", kind="violin", hue="merchantCategoryCode", aspect=2)

<img src="figures/transactionAmount_violinplot_by_isFraud_with_merchantCategoryCode.png" width="850"/>

In [34]:
countplot(df.select("isFraud", "cardPresent"), "isFraud", hue="cardPresent", stat="proportion")
countplot(df.select("isFraud", "cardPresent").where(df.isFraud == True), "cardPresent", stat="proportion")
countplot(df.select("isFraud", "cardPresent").where(df.isFraud == False), "cardPresent", stat="proportion")

<img src="figures/isFraud_barchart_with_cardPresent.png" width="330"/><img src="figures/cardPresent_barchart_isFraud.png" width="330"/><img src="figures/cardPresent_barchart_isnotFraud.png" width="330"/>

The three plots above show that while legitimate transactions have roughly equal likelihood of being performed with or without the card present, fraudulent transactions happend predominantly without the card present.